# 🧠 Advanced Feature Engineering: Reading Between the Lines

이 노트는 외부 데이터를 전혀 사용하지 않고, 주어진 데이터(`train.csv`)의 **숨겨진 맥락(Context)**을 찾아내는 과정입니다.

### 🎯 핵심 목표
1. **Game State (경기 상태)**: 현재 이기고 있는가? 지고 있는가? `score_difference`
2. **Player Persona (선수 성향)**: 이 선수는 평소에 어떤 플레이를 하는가? `player_embedding`
3. **Time Context (시간 맥락)**: 종료가 얼마나 남았는가? `time_remaining`

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

# 데이터 로드
train = pd.read_csv('./open_track1/train.csv')
match_info = pd.read_csv('./open_track1/match_info.csv')
print("Data Loaded")

## 1. Score Reconstruction (실시간 점수 복원)
데이터에는 결과 점수만 있고, **'그 순간의 점수'**가 없습니다.
경기 로그를 시간순으로 훑으면서 `Goal` 이벤트를 찾아 점수를 복원합니다.

In [ ]:
def calculate_game_states(df, match_info):
    # 결과 저장을 위한 딕셔너리 (game_id -> {time_seconds -> score_diff})
    # 하지만 더 효율적으로: game_id 별로 그룹화하여 처리
    
    df = df.sort_values(['game_id', 'period_id', 'time_seconds']).reset_index(drop=True)
    
    # 이벤트가 'Shot'이고 result_name이 'Goal'인 경우를 찾음
    df['is_goal'] = (df['type_name'] == 'Shot') & (df['result_name'] == 'Goal')
    
    # 홈/어웨이 구분을 위해 match_info 병합 (필요하다면)
    # 여기서는 간단하게 'team_id'가 바뀌는 것을 감지하거나,
    # 그냥 '해당 시점까지의 양 팀 득점 합'을 구할 수도 있지만,
    # '내가 이기고 있나?'가 중요하므로, 현재 player의 team_id 기준 득점 - 상대팀 득점이 필요함.
    
    # 이 로직은 복잡하므로 단순화: 
    # period 1, 2 내에서 누적 득점을 계산.
    
    # 1. 각 게임별, 팀별 누적 득점 계산
    # cumsum을 쓰기 위해 game_id, team_id 별로 정렬 필요...하지만 시간 순서가 섞이면 안됨.
    
    # 좋은 방법: 
    # 골이 터진 순간을 기록하고, merge_asof 로 붙이거나
    # iterrows 를 돌면서 상태를 추적 (느림)
    # 또는 GroupBy + Cumsum 활용
    
    # 여기서는 '팀별 누적 골'을 계산
    # 주의: 자책골(Own Goal) 처리 등이 이슈가 될 수 있으나, 데이터에 명시된 'Goal'만 믿고 감.
    
    # 'Goal' 이벤트에 1 할당, 나머지는 0
    df['goal_val'] = df['is_goal'].astype(int)
    
    # 게임별, 팀별 누적 합 (이것은 '해당 팀의 현재 점수'가 됨)
    # 하지만 우리는 '현재 시점'의 점수를 모든 행에 붙여야 함.
    # 즉, Goal이 발생한 행 이후부터 점수가 올라가야 함.
    
    score_cols = df.groupby(['game_id', 'team_id'])['goal_val'].cumsum()
    # 주의: cumsum은 해당 행을 포함하므로, 'Goal' 이벤트 발생 직후부터 적용되어야 하는지, 직전인지 판단.
    # 축구에서는 골이 들어간 직후 킥오프부터 스코어가 반영됨. 
    # 현재 행(Goal)은 아직 득점 전 상황의 플레이인가? -> 슈팅은 득점 전임.
    # 따라서 cumsum 결과에서 자기 자신 값을 빼거나, shift(1)을 해줘야 '현재 상황'임.
    
    df['current_team_score'] = df.groupby(['game_id', 'team_id'])['goal_val'].shift(1).fillna(0)
    
    # 상대팀 점수는 어떻게 알지? -> game_id 안에서 '총 골 수' - '내 팀 골 수'
    # 전체 골 누적 (게임별)
    df['game_total_goals'] = df.groupby('game_id')['goal_val'].shift(1).fillna(0)
    
    df['opp_team_score'] = df['game_total_goals'] - df['current_team_score']
    
    df['score_diff'] = df['current_team_score'] - df['opp_team_score']
    
    return df[['game_id', 'period_id', 'time_seconds', 'score_diff', 'current_team_score', 'opp_team_score']]

print("Calculating Game States...")
train_states = calculate_game_states(train.copy(), match_info)
print("Done.")
train_states.head()

## 2. Player Stats (선수 성향)
각 선수가 패스할 때 평균적으로 얼마나 멀리 보내는지, 어느 방향으로 보내는지 학습합니다.

In [ ]:
def calculate_player_stats(df):
    # 패스 이벤트만 추출
    passes = df[df['type_name'] == 'Pass'].copy()
    
    # 거리 계산
    passes['dx'] = passes['end_x'] - passes['start_x']
    passes['dy'] = passes['end_y'] - passes['start_y']
    passes['distance'] = np.sqrt(passes['dx']**2 + passes['dy']**2)
    passes['angle'] = np.arctan2(passes['dy'], passes['dx'])
    
    # 선수별 평균 거리, 평균 각도, 패스 횟수
    player_stats = passes.groupby('player_id').agg(
        avg_pass_dist=('distance', 'mean'),
        avg_pass_angle=('angle', 'mean'),
        pass_count=('distance', 'count')
    ).reset_index()
    
    return player_stats

print("Calculating Player Stats...")
player_features = calculate_player_stats(train)
print("Done.")
player_features.head()

## 3. Merge & Save
만들어진 Feature들을 저장하고, 다음 모델링 단계에서 사용할 수 있도록 준비합니다.

In [ ]:
import os

OUT_DIR = './data_preprocess'
os.makedirs(OUT_DIR, exist_ok=True)

train_states.to_csv(os.path.join(OUT_DIR, 'train_game_states.csv'), index=False)
player_features.to_csv(os.path.join(OUT_DIR, 'player_features.csv'), index=False)

print(f"Files saved to {OUT_DIR}")